# Пакет `pandas`: группировка, агрегирование, сводные таблицы

---

**Источники:**

[Сводные таблицы в Python](http://datareview.info/article/svodnyie-tablitsyi-v-python/)

[Group by: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

[Pandas groupby: 13 Functions To Aggregate](https://cmdlinetips.com/2019/10/pandas-groupby-13-functions-to-aggregate/)

[Aggregation and Grouping](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html)

[pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)

---

## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

!pip -V

pip 20.3 from /home/ira/anaconda3/envs/LevelUp_DataScience/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
# !conda install pandas -y

In [3]:
import pandas as pd

pd.__version__

'1.2.1'

## Загрузка данных

Для примера возьмем [данные (data set) Gapminder World](https://www.kaggle.com/tklimonova/gapminder-datacamp-2007).

In [4]:
df = pd.read_csv('./../../data/gapminder_full.csv')

# показать все колонки
pd.options.display.max_columns = None

# отобразить первые 5 и последние 5 строк
df

,country,year,population,continent,life_exp,gdp_cap
0,Afghanistan,1952,8425333,Asia,28.801,779.445314
1,Afghanistan,1957,9240934,Asia,30.332,820.853030
2,Afghanistan,1962,10267083,Asia,31.997,853.100710
3,Afghanistan,1967,11537966,Asia,34.020,836.197138
4,Afghanistan,1972,13079460,Asia,36.088,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,1987,9216418,Africa,62.351,706.157306
1700,Zimbabwe,1992,10704340,Africa,60.377,693.420786
1701,Zimbabwe,1997,11404948,Africa,46.809,792.449960
1702,Zimbabwe,2002,11926563,Africa,39.989,672.038623


In [5]:
# показать первые 5 строк
df.head()

,country,year,population,continent,life_exp,gdp_cap
0,Afghanistan,1952,8425333,Asia,28.801,779.445314
1,Afghanistan,1957,9240934,Asia,30.332,820.853030
2,Afghanistan,1962,10267083,Asia,31.997,853.100710
3,Afghanistan,1967,11537966,Asia,34.020,836.197138
4,Afghanistan,1972,13079460,Asia,36.088,739.981106


## Группировка

За группировку отвечает метод `pandas.groupby`.

Функция `Pandas` `groupby` позволяет легко выполнять парадигму анализа данных **разделить-применить-объединить (Split-Apply-Combine)**.

По сути, с помощью  `groupby` можно разбить фрейм данных на более мелкие группы, используя одну или несколько переменных. 


## Функции для агрегирования

`Pandas` имеет ряд **агрегирующих** функций, которые уменьшают размер сгруппированного объекта.

- `mean()`: среднее значение для каждой группы
- `sum()`: сумма значений для каждой группы
- `size()`: размер (количество строк) каждой группы
- `count()`: количество значений в каждой группе
- `std()`: стандартное отклонение (**st**andar**d** deviation) для каждой группы
- `var()`: дисперсия (**var**iance) для каждой группы
- `sem()`: стандартная ошибка среднего (**S**tandard **E**rror of the **M**ean) для каждой группы
- `describe()`: описательная статистика
- `first()`: первое из значений группы
- `last()`: последнее из значений группы
- `nth()`: n-е значение или подмножество, если n - список
- `min()`: минимальное значение для каждой группы
- `max()`: максимальное значение для каждой группы

### `mean()`: среднее значение для каждой группы

In [6]:
# среднее значение в каждой колонке по континетнам
df.groupby(["continent"]).mean()

,year,population,life_exp,gdp_cap
continent,,,,
Africa,1979.5,9.916003e+06,48.865330,2193.754578
Americas,1979.5,2.450479e+07,64.658737,7136.110356
Asia,1979.5,7.703872e+07,60.064903,7902.150428
Europe,1979.5,1.716976e+07,71.903686,14469.475533
Oceania,1979.5,8.874672e+06,74.326208,18621.609223


In [7]:
# можно НЕ использовать [], если только одна колонка

# среднее значение в каждой колонке по континетнам
df.groupby("continent").mean()

,year,population,life_exp,gdp_cap
continent,,,,
Africa,1979.5,9.916003e+06,48.865330,2193.754578
Americas,1979.5,2.450479e+07,64.658737,7136.110356
Asia,1979.5,7.703872e+07,60.064903,7902.150428
Europe,1979.5,1.716976e+07,71.903686,14469.475533
Oceania,1979.5,8.874672e+06,74.326208,18621.609223


In [8]:
# среднее значение численности населения по континетнам
df.groupby("continent")["population"].mean()

continent
Africa      9.916003e+06
Americas    2.450479e+07
Asia        7.703872e+07
Europe      1.716976e+07
Oceania     8.874672e+06
Name: population, dtype: float64

### `sum()`: сумма значений для каждой группы

In [9]:
# суммарное значение для каждой колонки по каждому континенту
df.groupby("continent").sum()

,year,population,life_exp,gdp_cap
continent,,,,
Africa,1235208,6187585961,30491.96600,1.368903e+06
Americas,593850,7351438499,19397.62100,2.140833e+06
Asia,783882,30507333901,23785.70168,3.129252e+06
Europe,712620,6181115304,25885.32700,5.209011e+06
Oceania,47508,212992136,1783.82900,4.469186e+05


In [10]:
# суммарное значение численности населения по каждому континенту
df.groupby("continent").population.sum()

continent
Africa       6187585961
Americas     7351438499
Asia        30507333901
Europe       6181115304
Oceania       212992136
Name: population, dtype: int64

### `size()`: размер (количество строк) каждой группы

In [11]:
# количество строк для каждого континента
# пропущенные значения (nan, null...) считаются
df.groupby("continent").size()

continent
Africa      624
Americas    300
Asia        396
Europe      360
Oceania      24
dtype: int64

### `count()`: количество значений в каждой группе

По сути, это та же функция агрегирования, что и `size`, НО *игнорируются любые отсутствующие значения*.

Если фрейм данных не имеет пропущенных значений, поэтому результаты обеих функций одинаковы.

In [12]:
# количество значений для каждой колонки для каждого континента
# пропущенные значения (nan, null...) НЕ считаются
df.groupby("continent").count()

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,624,624,624,624,624
Americas,300,300,300,300,300
Asia,396,396,396,396,396
Europe,360,360,360,360,360
Oceania,24,24,24,24,24


In [13]:
# количество значений о численности населения для каждого континента
# пропущенные значения (nan, null...) НЕ считаются
df.groupby("continent").population.count()

continent
Africa      624
Americas    300
Asia        396
Europe      360
Oceania      24
Name: population, dtype: int64

### `std()`: стандартное отклонение (standard deviation) для каждой группы

In [14]:
# стандартное отклонение для каждой колонки для каждого континента
df.groupby("continent").std()

,year,population,life_exp,gdp_cap
continent,,,,
Africa,17.274110,1.549092e+07,9.150210,2827.929863
Americas,17.289102,5.097943e+07,9.345088,6396.764112
Asia,17.282097,2.068852e+08,11.864532,14045.373112
Europe,17.284285,2.051944e+07,5.433178,9355.213498
Oceania,17.631494,6.506342e+06,3.795611,6358.983321


In [15]:
# стандартное отклонение численности населения для каждого континента
df.groupby("continent").population.std()

continent
Africa      1.549092e+07
Americas    5.097943e+07
Asia        2.068852e+08
Europe      2.051944e+07
Oceania     6.506342e+06
Name: population, dtype: float64

### `var()`: дисперсия для каждой группы

In [16]:
# дисперсия для каждой колонки для каждого континента
df.groupby("continent").var()

,year,population,life_exp,gdp_cap
continent,,,,
Africa,298.394864,2.399687e+14,83.726347,7.997187e+06
Americas,298.913043,2.598902e+15,87.330670,4.091859e+07
Asia,298.670886,4.280149e+16,140.767108,1.972725e+08
Europe,298.746518,4.210473e+14,29.519421,8.752002e+07
Oceania,310.869565,4.233249e+13,14.406663,4.043667e+07


In [17]:
# дисперсия численности населения для каждого континента
df.groupby("continent").population.var()

continent
Africa      2.399687e+14
Americas    2.598902e+15
Asia        4.280149e+16
Europe      4.210473e+14
Oceania     4.233249e+13
Name: population, dtype: float64

### `sem()`: стандартная ошибка среднего (Standard Error of the Mean) для каждой группы

In [18]:
# стандартная ошибка среднего для каждой колонки для каждого континента
df.groupby("continent").sem()

,year,population,life_exp,gdp_cap
continent,,,,
Africa,0.691518,6.201332e+05,0.366302,113.207797
Americas,0.998187,2.943299e+06,0.539539,369.317348
Asia,0.868458,1.039637e+07,0.596215,705.806555
Europe,0.910962,1.081469e+06,0.286354,493.063044
Oceania,3.599014,1.328102e+06,0.774776,1298.022035


In [19]:
# стандартная ошибка среднего численности населения для каждого континента
df.groupby("continent").population.sem()

continent
Africa      6.201332e+05
Americas    2.943299e+06
Asia        1.039637e+07
Europe      1.081469e+06
Oceania     1.328102e+06
Name: population, dtype: float64

### `describe()`: описательная статистика

Краткая сводка по значениям в каждой группе.

`describe()` вычисляет количество значений, среднее, стандартное, минимальное значение, максимальное значение и значение в нескольких процентилях.

In [20]:
# статистика для каждой колонки для каждого континента
df.groupby("continent").describe()

year                                                               \
           count    mean        std     min      25%     50%      75%     max   
continent                                                                       
Africa     624.0  1979.5  17.274110  1952.0  1965.75  1979.5  1993.25  2007.0   
Americas   300.0  1979.5  17.289102  1952.0  1965.75  1979.5  1993.25  2007.0   
Asia       396.0  1979.5  17.282097  1952.0  1965.75  1979.5  1993.25  2007.0   
Europe     360.0  1979.5  17.284285  1952.0  1965.75  1979.5  1993.25  2007.0   
Oceania     24.0  1979.5  17.631494  1952.0  1965.75  1979.5  1993.25  2007.0   

          population                                                     \
               count          mean           std        min         25%   
continent                                                                 
Africa         624.0  9.916003e+06  1.549092e+07    60011.0  1342075.00   
Americas       300.0  2.450479e+07  5.097943e+07   662850.0  2962358.75   
Asia           396.0  7.703872e+07  2.068852e+08   120447.0  3844393.00   
Europe         360.0  1.716976e+07  2.051944e+07   147962.0  4331500.00   
Oceania         24.0  8.874672e+06  6.506342e+06  1994794.0  3199212.50   

                                                 life_exp             \
                  50%          75%           max    count       mean   
continent                                                              
Africa      4579311.0  10801489.75  1.350312e+08    624.0  48.865330   
Americas    6227510.0  18340309.00  3.011399e+08    300.0  64.658737   
Asia       14530830.5  46300348.00  1.318683e+09    396.0  60.064903   
Europe      8551125.0  21802867.00  8.240100e+07    360.0  71.903686   
Oceania     6403491.5  14351625.00  2.043418e+07     24.0  74.326208   

                                                                  gdp_cap  \
                 std     min       25%      50%       75%     max   count   
continent                                                                   
Africa      9.150210  23.599  42.37250  47.7920  54.41150  76.442   624.0   
Americas    9.345088  37.579  58.41000  67.0480  71.69950  80.653   300.0   
Asia       11.864532  28.801  51.42625  61.7915  69.50525  82.603   396.0   
Europe      5.433178  43.585  69.57000  72.2410  75.45050  81.757   360.0   
Oceania     3.795611  69.120  71.20500  73.6650  77.55250  81.235    24.0   

                                                                   \
                   mean           std           min           25%   
continent                                                           
Africa      2193.754578   2827.929863    241.165876    761.247010   
Americas    7136.110356   6396.764112   1201.637154   3427.779072   
Asia        7902.150428  14045.373112    331.000000   1056.993223   
Europe     14469.475533   9355.213498    973.533195   7213.085036   
Oceania    18621.609223   6358.983321  10039.595640  14141.858697   

                                                     
                    50%           75%           max  
continent                                            
Africa      1192.138217   2377.417422   21951.21176  
Americas    5465.509853   7830.210416   42951.65309  
Asia        2646.786844   8549.255654  113523.13290  
Europe     12081.749115  20461.386162   49357.19017  
Oceania    17983.303955  22214.117110   34435.36744

In [21]:
# статистика по численности населения для каждого континента
df.groupby("continent").population.describe()

,count,mean,std,min,25%,50%,75%,max
continent,,,,,,,,
Africa,624.0,9.916003e+06,1.549092e+07,60011.0,1342075.00,4579311.0,10801489.75,1.350312e+08
Americas,300.0,2.450479e+07,5.097943e+07,662850.0,2962358.75,6227510.0,18340309.00,3.011399e+08
Asia,396.0,7.703872e+07,2.068852e+08,120447.0,3844393.00,14530830.5,46300348.00,1.318683e+09
Europe,360.0,1.716976e+07,2.051944e+07,147962.0,4331500.00,8551125.0,21802867.00,8.240100e+07
Oceania,24.0,8.874672e+06,6.506342e+06,1994794.0,3199212.50,6403491.5,14351625.00,2.043418e+07


### `first()`: первое из значений группы

Получает значение из первой строки в каждой группе.

In [22]:
# первое значение для каждой колонки для каждого континента
df.groupby("continent").first()

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,Algeria,1952,9279525,43.077,2449.008185
Americas,Argentina,1952,17876956,62.485,5911.315053
Asia,Afghanistan,1952,8425333,28.801,779.445314
Europe,Albania,1952,1282697,55.230,1601.056136
Oceania,Australia,1952,8691212,69.120,10039.595640


In [23]:
# первое значение численности населения для каждого континента
df.groupby("continent").population.first()

continent
Africa       9279525
Americas    17876956
Asia         8425333
Europe       1282697
Oceania      8691212
Name: population, dtype: int64

### `last()`: последнее из значений группы

Получает значение из последней строки в каждой группе.

In [24]:
# последнее значение для каждой колонки для каждого континента
df.groupby("continent").last()

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,Zimbabwe,2007,12311143,43.487,469.709298
Americas,Venezuela,2007,26084662,73.747,11415.805690
Asia,"Yemen, Rep.",2007,22211743,62.698,2280.769906
Europe,United Kingdom,2007,60776238,79.425,33203.261280
Oceania,New Zealand,2007,4115771,80.204,25185.009110


In [25]:
# последнее значение численности населения для каждого континента
df.groupby("continent").population.last()

continent
Africa      12311143
Americas    26084662
Asia        22211743
Europe      60776238
Oceania      4115771
Name: population, dtype: int64

### `nth()`: n-е значение или подмножество, если n - список

In [26]:
# 15 значение для каждой колонки для каждого континента
df.groupby("continent").nth(15)

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,Angola,1967,5247469,35.985,5522.776375
Americas,Bolivia,1967,4040665,45.032,2586.886053
Asia,Bahrain,1967,202182,59.923,14804.672700
Europe,Austria,1967,7376998,70.140,12834.602400
Oceania,New Zealand,1967,2728150,71.520,14463.918930


In [27]:
# 1, 2 и 15 значение для каждой колонки для каждого континента
df.groupby("continent").nth([1, 2, 15])

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,Algeria,1957,10270856,45.685,3013.976023
Africa,Algeria,1962,11000948,48.303,2550.816880
Africa,Angola,1967,5247469,35.985,5522.776375
Americas,Argentina,1957,19610538,64.399,6856.856212
Americas,Argentina,1962,21283783,65.142,7133.166023
Americas,Bolivia,1967,4040665,45.032,2586.886053
Asia,Afghanistan,1957,9240934,30.332,820.853030
Asia,Afghanistan,1962,10267083,31.997,853.100710
Asia,Bahrain,1967,202182,59.923,14804.672700


In [28]:
# 15 значение численности населения для каждого континента
df.groupby("continent").population.nth(15)

continent
Africa      5247469
Americas    4040665
Asia         202182
Europe      7376998
Oceania     2728150
Name: population, dtype: int64

In [29]:
# 1, 2 и 15 значение численности населения для каждого континента
df.groupby("continent").population.nth([1, 2, 15])

continent
Africa      10270856
Africa      11000948
Africa       5247469
Americas    19610538
Americas    21283783
Americas     4040665
Asia         9240934
Asia        10267083
Asia          202182
Europe       1476505
Europe       1728137
Europe       7376998
Oceania      9712569
Oceania     10794968
Oceania      2728150
Name: population, dtype: int64

### `min()`: минимальное значение для каждой группы

In [30]:
# минимальное значение для каждой колонки для каждого континента
df.groupby("continent").min()

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,Algeria,1952,60011,23.599,241.165876
Americas,Argentina,1952,662850,37.579,1201.637154
Asia,Afghanistan,1952,120447,28.801,331.000000
Europe,Albania,1952,147962,43.585,973.533195
Oceania,Australia,1952,1994794,69.120,10039.595640


In [31]:
# минимальное значение численности населения для каждого континента
df.groupby("continent").population.min()

continent
Africa        60011
Americas     662850
Asia         120447
Europe       147962
Oceania     1994794
Name: population, dtype: int64

### `max()`: максимальное значение для каждой группы

In [32]:
# максимальное значение для каждой колонки для каждого континента
df.groupby("continent").max()

,country,year,population,life_exp,gdp_cap
continent,,,,,
Africa,Zimbabwe,2007,135031164,76.442,21951.21176
Americas,Venezuela,2007,301139947,80.653,42951.65309
Asia,"Yemen, Rep.",2007,1318683096,82.603,113523.13290
Europe,United Kingdom,2007,82400996,81.757,49357.19017
Oceania,New Zealand,2007,20434176,81.235,34435.36744


In [33]:
# максимальное значение численности населения для каждого континента
df.groupby("continent").population.max()

continent
Africa       135031164
Americas     301139947
Asia        1318683096
Europe        82400996
Oceania       20434176
Name: population, dtype: int64

In [34]:
# доля выживших для каждого пола
df.groupby('Sex')[['Survived']].mean()

KeyError: 'Sex'

In [ ]:
# взаимосвязь между показателем выживаемости, полом и классом
df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean').unstack()

In [ ]:
# если без unstack()
df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean')

Хотя каждый шаг этой последовательности вполне понятен, тем не менее длинную строку кода достаточно трудно читать и использовать.

Подобные операции широко распространены, в связи с чем библиотека `pandas` имеет в своем составе специальный метод `pivot_table,` лаконично реализующий данный тип многомерной агрегации.

## Что такое сводная таблица?

**Сводная таблица (англ. Pivot table)** — инструмент обработки данных, служащий для их обобщения.


*Термин "сводная таблица" может быть знаком из `Microsoft Excel` или любым иным, предназначенным для обработки и анализа данных. 


В `pandas` сводные таблицы строятся через метод `DataFrame.pivot_table`.

In [ ]:
# взаимосвязь между показателем выживаемости, полом и классом
df.pivot_table(values='Survived', 
                       index='Sex', 
                       columns='Pclass')

In [ ]:
# посчитать сколько всего женщин и мужчин было в конкретном классе каюты
pvt = df.pivot_table(index=['Sex'], 
                             columns=['Pclass'], 
                             values='Name', 
                             aggfunc='count')

pvt

In [ ]:
# качестве индекса будет пол человека, 
# колонками станут значения из Pclass, 
# функцией агрегирования будет count по колонке Name.
pvt.loc['female', [1, 2, 3]]

## Многоуровневые сводные таблицы

Точно так же, как при использовании `groupby`, группирование в сводной таблице может иметь несколько уровней и задаваться посредством различных параметров.

In [ ]:
# разделить возраст на интервалы
age = pd.cut(df['Age'], [0, 18, 80])

age

In [ ]:
# взаимосвязь между 
# показателем выживаемости, полом, классом и возрастом
df.pivot_table(values='Survived', 
                       index=['Sex', age], 
                       columns='Pclass')

In [ ]:
# добавить информацию о стоимости билета (квантили):
fare = pd.qcut(df['Fare'], 2)

fare

In [ ]:
df.pivot_table(values='Survived', 
                       index=['Sex', age], 
                       columns=[fare, 'Pclass'],
                       aggfunc='count')

В результате получили четырехмерную агрегацию, демонстрирующую взаимосвязь между соответствующими величинами.

## Дополнительные параметры сводной таблицы

Полная [сигнатура метода `pivot_table`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) объекта `DataFrame` является следующей:

```
pandas.pivot_table(values=None, 
                   index=None, 
                   columns=None, 
                   aggfunc='mean', 
                   fill_value=None, 
                   margins=False, 
                   dropna=True, 
                   margins_name='All', 
                   observed=False)
```


Параметры **`fill_value`** и **`dropna`** задают способ обработки отсутствующих данных. 

Параметр **`aggfunc`** задает тип агрегации. По умолчанию его значение равно `mean`. Как и в случае `groupby`, тип агрегации можно задать либо с помощью предопределенной строки (например, `sum`, `mean`, `count`, `min`, `max` и др.), либо посредством функции, реализующей агрегацию (например, `np.sum()`, `min()`, `sum()` и др.).

Кроме того, параметр **`aggfunc`** может быть задан в виде словаря, *отображающего* столбцы на любые из желаемых значений, перечисленных выше:

In [ ]:
# в данном случае не задается параметр values, 
# так как values задается автоматически, 
# когда параметр aggfunc представлен в виде отображения
df.pivot_table(index='Sex', 
                       columns='Pclass', 
                       aggfunc={'Survived': sum, 
                                'Fare': 'mean'})

С помощью параметра **`margins`** можно вычислить обобщенные значения по каждой группе.

In [ ]:
df.pivot_table(values='Survived', 
                       index='Sex', 
                       columns='Pclass', 
                       aggfunc='count',
                       margins=True)